In [2]:
import pipeline as pp

In [186]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import pickle
from sklearn.externals.joblib import Memory
from census import Census
import requests
memory = Memory(location='/tmp', verbose=0)

In [17]:
pd.options.display.max_columns = 999

In [25]:
# make sure these column aren't numerical
lcs_coltypes = {
    'LICENSE ID': str,
    'ACCOUNT NUMBER': str,
    'WARD': str,
    'PRECINCT': str,
    'POLICE DISTRICT': str,
    'LICENSE CODE': str,
    'LICENSE NUMBER': str,
    'SSA': str
}

# date columns to parse
lcs_datecols = [
    'APPLICATION CREATED DATE', 
    'APPLICATION REQUIREMENTS COMPLETE', 
    'PAYMENT DATE',
    'LICENSE TERM START DATE',
    'LICENSE TERM EXPIRATION DATE',
    'LICENSE APPROVED FOR ISSUANCE',
    'DATE ISSUED',
    'LICENSE STATUS CHANGE DATE'
]

In [26]:
lcs = pp.read_data('data/Business_Licenses.csv', coltypes=lcs_coltypes, parse_dates=lcs_datecols)

/usr/local/anaconda3/envs/mlpp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (9,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [60]:
len(lcs)

968908

In [27]:
lcs.head()

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,LICENSE NUMBER,APPLICATION TYPE,APPLICATION CREATED DATE,APPLICATION REQUIREMENTS COMPLETE,PAYMENT DATE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,14,NaN,14-,8,1010,Limited Business License,NaN,NaN,22308,RENEW,NaT,2006-06-21,2006-08-10,N,2006-08-16,2007-08-15,2006-08-10,2006-08-11,AAI,NaT,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,25,28,25-28,1,1010,Limited Business License,911,Retail Sales of Clothing / Accessories / Shoes,1620668,RENEW,NaT,2016-03-15,2016-05-18,N,2016-05-16,2018-05-15,2016-05-18,2016-08-30,AAI,NaT,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)"
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,25,18,25-18,9,1625,Raffles,720,Not-For-Profit Selling Raffles for Prizes of $...,2368602,RENEW,NaT,2016-04-15,2016-06-21,N,2016-06-16,2017-06-15,2016-06-21,2016-06-22,AAC,2016-08-30,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)"
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,NaN,NaN,NaN,NaN,1010,Limited Business License,NaN,NaN,2060891,RENEW,NaT,2014-08-15,2016-04-01,N,2014-10-16,2016-10-15,2016-04-01,2016-04-01,AAI,NaT,38,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)"
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,10,25,10-25,4,1010,Limited Business License,NaN,NaN,1144216,RENEW,NaT,2007-03-23,2007-05-10,N,2007-05-16,2008-05-15,2007-05-10,2007-05-11,AAI,NaT,5,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)"


In [28]:
lcs.dtypes

ID                                           object
LICENSE ID                                   object
ACCOUNT NUMBER                               object
SITE NUMBER                                   int64
LEGAL NAME                                   object
DOING BUSINESS AS NAME                       object
ADDRESS                                      object
CITY                                         object
STATE                                        object
ZIP CODE                                     object
WARD                                         object
PRECINCT                                     object
WARD PRECINCT                                object
POLICE DISTRICT                              object
LICENSE CODE                                 object
LICENSE DESCRIPTION                          object
BUSINESS ACTIVITY ID                         object
BUSINESS ACTIVITY                            object
LICENSE NUMBER                               object
APPLICATION 

In [74]:
def gdf_from_latlong(df, lat, long):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[long], df[lat]))
    return gdf

In [57]:
lcs = gdf_from_latlong(lcs, lat='LATITUDE', long='LONGITUDE')

/usr/local/anaconda3/envs/mlpp/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Persisting input arguments took 75.54s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  """Entry point for launching an IPython kernel.


In [68]:
# save licenses gdf for later 
pickle.dump(lcs, open("pickle/lcs", "wb" ))

In [73]:
lcs

,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,WARD,PRECINCT,WARD PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE DESCRIPTION,BUSINESS ACTIVITY ID,BUSINESS ACTIVITY,LICENSE NUMBER,APPLICATION TYPE,APPLICATION CREATED DATE,APPLICATION REQUIREMENTS COMPLETE,PAYMENT DATE,CONDITIONAL APPROVAL,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION,geometry
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,14,NaN,14-,8,1010,Limited Business License,NaN,NaN,22308,RENEW,NaT,2006-06-21,2006-08-10,N,2006-08-16,2007-08-15,2006-08-10,2006-08-11,AAI,NaT,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)",POINT (-87.72395999700001 41.82531993)
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,25,28,25-28,1,1010,Limited Business License,911,Retail Sales of Clothing / Accessories / Shoes,1620668,RENEW,NaT,2016-03-15,2016-05-18,N,2016-05-16,2018-05-15,2016-05-18,2016-08-30,AAI,NaT,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)",POINT (-87.641590057 41.867338566)
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,25,18,25-18,9,1625,Raffles,720,Not-For-Profit Selling Raffles for Prizes of $...,2368602,RENEW,NaT,2016-04-15,2016-06-21,N,2016-06-16,2017-06-15,2016-06-21,2016-06-22,AAC,2016-08-30,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)",POINT (-87.63873424399999 41.850842944)
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,NaN,NaN,NaN,NaN,1010,Limited Business License,NaN,NaN,2060891,RENEW,NaT,2014-08-15,2016-04-01,N,2014-10-16,2016-10-15,2016-04-01,2016-04-01,AAI,NaT,38,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)",POINT (-87.67858578000001 41.951315556)
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,10,25,10-25,4,1010,Limited Business License,NaN,NaN,1144216,RENEW,NaT,2007-03-23,2007-05-10,N,2007-05-16,2008-05-15,2007-05-10,2007-05-11,AAI,NaT,5,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)",POINT (-87.551366466 41.728621736)
5,2488345-20160830,2488345,409564,1,"BURKS HEATING AND COOLING SOLUTIONS, LLC","BURKS HEATING AND COOLING SOLUTIONS, LLC",5722 S WOOD ST 1ST,CHICAGO,IL,60636,15,35,15-35,7,4404,Regulated Business License,674,Home Repair Services (Home Occ),2488345,ISSUE,2016-08-30,2016-08-30,2016-08-30,N,2016-08-30,2018-09-15,2016-08-30,2016-08-30,AAI,NaT,NaN,41.789617,-87.669472,"(41.78961679000226, -87.66947199630827)",POINT (-87.669471996 41.78961679)
6,22308-20030816,1479006,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,14,NaN,14-,8,1010,Limited Business License,NaN,NaN,22308,RENEW,NaT,2004-03-05,2004-03-22,N,2003-08-16,2004-08-15,2004-03-25,2004-04-16,AAI,NaT,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)",POINT (-87.72395999700001 41.82531993)
7,2354707-20160916,2482568,393149,1,ANGELINE R. MC CARTHY,ANGELINE R. MC CARTHY,17707 S HARLEM ST 1,TINLEY PARK,IL,60477,NaN,NaN,NaN,NaN,4406,Peddler License,915,"Street Performer, Moving Along the Public Way ...",2354707,RENEW,NaT,2016-07-15,2016-08-30,N,2016-09-16,2018-09-15,NaT,2016-08-30,AAI,NaT,NaN,NaN,NaN,NaN,POINT (nan nan)
8,22308-20010816,1151715,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,14,NaN,14-,8,1010,Limited Business License,NaN,NaN,22308,RENEW,NaT,2001-06-22,2001-08-10,N,2001-08-16,2002-08-15,2001-08-10,2004-03-05,AAI,NaT,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)",POINT (-87.72395999700001 41.82531993)
9,1144216-20030516,1338337,147,63,WALGREE

In [ ]:
# get acs 5-year estimate for each year (block group, cook county) with census api

# current problems:
# 1) earliest 2009, latest 2017 -- doesn't cover all of our periods
# 2) can't 2010-2012 using API (error: unknown/unsupported geography heirarchy)

In [148]:
# ACS variables
# should add education level variables
# potentially also work-residence distance
var_dic = {
    'NAME': 'NAME',
    'B01003_001E': 'population_total',
    'B02001_002E': 'population_white',
    'B02001_003E': 'population_black',
    'B02001_004E': 'population_american_indian_and_alaskan_native',
    'B02001_005E': 'population_asian',
    'B02001_006E': 'population_native_hawaiian',
    'B02001_007E': 'population_some_other_race_alone',
    'B02001_008E': 'population_two_or_more_races',
    'B02001_009E': 'population_two_races_including_some_other_race',
    'B02001_010E': 'population_two_races_excluding_some_other_race_and_three_or_more_races',
    'B19013_001E': 'median_income'
}

In [130]:
# initialize 
api_key = '68c71cf3327ee04e568b590529287b186460bdd4'
c = Census(api_key)
var_list = tuple([v for v in var_dic.keys()])

In [172]:
# query census api

acs_results = {}
for year in range(2013, 2018):
    print("obtaining data from {}...".format(year))
    acs_results[year] = c.acs5.get(var_list, 
                                   {'for': 'block group:*', 
                                    'in': 'state:17 county:031'}, 
                                   year=year)

obtaining data from 2013...
obtaining data from 2014...
obtaining data from 2015...
obtaining data from 2016...
obtaining data from 2017...


In [184]:
# for debugging acs5 2010-12 issue by manually constructing url
# this url doesn't work, but changing the date to 2013 works

geo_dic = {'for': 'block group:*',
           'in': 'state:017 county:031'}

def get_acs5(year, var_dic, geo_dic):
    '''to manually construct url'''
    # add year
    url = 'https://api.census.gov/data/{}/acs/acs5?get='.format(year)
    # add fields
    vars = ','.join(var_dic)
    url += vars
    
    # add geographies
    for key, criteria in geo_dic.items():
        url += "&"
        url += '{}={}'.format(key, criteria)   
    return url

get_acs5(2012, var_dic, geo_dic)

'https://api.census.gov/data/2012/acs/acs5?get=NAME,B01003_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,B02001_009E,B02001_010E,B19013_001E&for=block group:*&in=state:017 county:031'